#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: February 14th, 2021*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os
import time
import spacy


TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU

Colaboratory uses either a Nvidia T4 GPU or an Nvidia K80 GPU. The T4 is slightly faster than the old K80 for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Mon Apr  5 04:05:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [3]:
#gpt2.download_gpt2(model_name="124M")
nlp = spacy.load('en_core_web_lg')


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [6]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [7]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "en_tech_train.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
import tensorflow as tf

tf.reset_default_graph()
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.07s/it]


dataset has 316436 tokens
Training...
[10 | 29.59] loss=4.22 avg=4.22
[20 | 54.06] loss=4.15 avg=4.18
[30 | 77.82] loss=4.07 avg=4.15
[40 | 101.99] loss=3.97 avg=4.10
[50 | 126.46] loss=4.01 avg=4.08
[60 | 150.57] loss=3.85 avg=4.04
[70 | 174.60] loss=3.73 avg=4.00
[80 | 198.84] loss=3.77 avg=3.97
[90 | 223.12] loss=3.59 avg=3.92
[100 | 247.37] loss=3.40 avg=3.87
[110 | 271.63] loss=3.51 avg=3.84
[120 | 295.86] loss=3.41 avg=3.80
[130 | 320.07] loss=3.70 avg=3.79
[140 | 344.21] loss=3.34 avg=3.76
[150 | 368.32] loss=3.73 avg=3.75
[160 | 392.40] loss=3.04 avg=3.71
[170 | 416.55] loss=3.01 avg=3.66
[180 | 440.76] loss=3.23 avg=3.63
[190 | 464.96] loss=3.27 avg=3.61
[200 | 489.21] loss=2.88 avg=3.57
======== SAMPLE 1 ========
 the world because we all know what happened to the worlds most wanted super heroes
the game was produced in the us and was created by a team of english african american and african americantalian filmmakers
the film was a box office hit with its portrayal of the lif

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [4]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [5]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [18]:
import random

file = open("/content/drive/MyDrive/en_tech_test.txt", "rb")
test_set = file.read().decode(encoding='utf-8')
processed_test_set = test_set.split("\n")
processed_test_set=random.sample(processed_test_set,100)
input_size =[1, 2, 3, 4, 5, 10, 15, 20]
dataset={}
for size in input_size:
    dataset[size]=[];
for line in processed_test_set:
    line=line.split()
    for size in input_size:
        if len(line)>=size+1:
           dataset[size].append(line[:size+1])
import numpy as np
import matplotlib as plt

#for size in input_size:
    #dataset[size]=np.array(dataset[size])
for key,value in dataset.items():
    for line in value:
        a=' '.join(line[:-1])
        del line[:-1]
        line.insert(0,a)

print(len(dataset[20]))
#print(dataset)


48


In [6]:
#prefix="Chinese"
#!python -m spacy download en_core_web_lg

dataset={1: [['highprofile', 'web'], ['nbc', 'alleged'], ['equally', 'it'], ['studies', 'have'], ['but', 'what'], ['from', '1'], ['in', 'the'], ['these', 'were'], ['each', 'one'], ['symbian', 'phones'], ['any', 'proposals'], ['while', 'not'], ['although', 'the'], ['the', 'band'], ['they', 'did'], ['getting', 'the'], ['the', 'canadian'], ['as', 'we'], ['after', 'the'], ['the', 'piracy'], ['highspeed', 'downlink'], ['earlier', 'in'], ['she', 'called'], ['this', 'last'], ['among', 'current'], ['many', 'other'], ['the', 'track'], ['wendy', 'ide'], ['if', 'the'], ['moreno', 'is'], ['the', 'plates'], ['the', 'thriller'], ['so', 'what'], ['the', 'complaints'], ['shochiku', 'film'], ['some', 'of'], ['search', 'engines'], ['she', 'said'], ['it', 'beats'], ['davis', 'was'], ['khan', 'was'], ['more', 'than'], ['the', 'next'], ['the', 'motion'], ['mr', 'wong'], ['the', 'images'], ['british', 'tv'], ['mainstream', 'radio'], ['he', 'wrote'], ['william', 'hill']], 2: [['highprofile web', 'portals'], ['nbc alleged', 'that'], ['equally it', 'may'], ['studies have', 'shown'], ['but what', 'has'], ['from 1', 'january'], ['in the', 'second'], ['these were', 'considered'], ['each one', 'has'], ['symbian phones', 'produced'], ['any proposals', 'to'], ['while not', 'always'], ['although the', 'us'], ['the band', 'are'], ['they did', 'not'], ['getting the', 'government'], ['the canadian', 'music'], ['as we', 'move'], ['after the', 'controversy'], ['the piracy', 'group'], ['highspeed downlink', 'packet'], ['earlier in', 'his'], ['she called', 'her'], ['this last', 'category'], ['among current', 'trends'], ['many other', 'firms'], ['the track', 'has'], ['wendy ide', 'the'], ['if the', 'decision'], ['moreno is', 'also'], ['the plates', 'decorating'], ['the thriller', 'shot'], ['so what', 'does'], ['the complaints', 'about'], ['shochiku film', 'studios'], ['some of', 'the'], ['search engines', 'are'], ['she said', 'she'], ['it beats', 'the'], ['davis was', 'also'], ['khan was', 'involved'], ['more than', '120000'], ['the next', 'major'], ['the motion', 'picture'], ['mr wong', 'eventually'], ['the images', 'have'], ['british tv', 'viewers'], ['mainstream radio', 'stations'], ['he wrote', 'ring'], ['william hill', 'made']], 3: [['highprofile web portals', 'such'], ['nbc alleged that', 'boxing'], ['equally it may', 'mean'], ['studies have shown', 'that'], ['but what has', 'not'], ['from 1 january', 'a'], ['in the second', 'ms'], ['these were considered', 'to'], ['each one has', 'involved'], ['symbian phones produced', 'by'], ['any proposals to', 'extend'], ['while not always', 'inventing'], ['although the us', 'leads'], ['the band are', 'also'], ['they did not', 'tell'], ['getting the government', 'even'], ['the canadian music', 'manager'], ['as we move', 'from'], ['after the controversy', 'last'], ['the piracy group', 'called'], ['highspeed downlink packet', 'access'], ['earlier in his', 'career'], ['she called her', 'service'], ['this last category', 'includes'], ['among current trends', 'it'], ['many other firms', 'have'], ['the track has', 'become'], ['wendy ide the', 'times'], ['if the decision', 'on'], ['moreno is also', 'nominated'], ['the plates decorating', 'the'], ['the thriller shot', 'straight'], ['so what does', 'this'], ['the complaints about', 'the'], ['shochiku film studios', 'which'], ['some of the', 'contributing'], ['search engines are', 'often'], ['she said she', 'was'], ['it beats the', 'ipod'], ['davis was also', 'a'], ['khan was involved', 'in'], ['more than 120000', 'trade'], ['the next major', 'film'], ['the motion picture', 'association'], ['mr wong eventually', 'hit'], ['the images have', 'three'], ['british tv viewers', 'lead'], ['mainstream radio stations', 'were'], ['he wrote ring', 'of'], ['william hill made', 'the']], 4: [['highprofile web portals such', 'as'], ['nbc alleged that boxing', 'regulations'], ['equally it may mean', 'hollywood'], ['studies have shown that', 'organisations'], ['but what has not', 'been'], ['from 1 january a', 'new'], ['in the second ms', 'love'], ['these were considered to', 'be'], ['each one has involved', 'the'], ['symbian phones produced by', 'sony'], ['any proposals to extend', 'existing'], ['while not always inventing', 'new'], ['although the us leads', 'in'], ['the band are also', 'working'], ['they did not tell', 'me'], ['getting the government even', 'more'], ['the canadian music manager', 'who'], ['as we move from', 'standard'], ['after the controversy last', 'year'], ['the piracy group called', 'itself'], ['highspeed downlink packet access', 'hsdpa'], ['earlier in his career', 'melcher'], ['she called her service', 'provider'], ['this last category includes', 'the'], ['among current trends it', 'highlighted'], ['many other firms have', 'released'], ['the track has become', 'the'], ['wendy ide the times', 'film'], ['if the decision on', 'the'], ['moreno is also nominated', 'for'], ['the plates decorating the', 'branches'], ['the thriller shot straight', 'to'], ['so what does this', 'mean'], ['the complaints about the', 'star'], ['shochiku film studios which', 'was'], ['some of the contributing', 'artists'], ['search engines are often', 'the'], ['she said she was', 'horrified'], ['it beats the ipod', 'into'], ['davis was also a', 'prominent'], ['khan was involved in', 'controversy'], ['more than 120000 trade', 'professionals'], ['the next major film', 'award'], ['the motion picture association', 'of'], ['mr wong eventually hit', 'upon'], ['the images have three', 'dimensions'], ['british tv viewers lead', 'the'], ['mainstream radio stations were', 'not'], ['he wrote ring of', 'fire'], ['william hill made the', 'move']], 5: [['highprofile web portals such as', 'google'], ['nbc alleged that boxing regulations', 'had'], ['equally it may mean hollywood', 'studios'], ['studies have shown that organisations', 'working'], ['but what has not been', 'explored'], ['from 1 january a new', 'law'], ['in the second ms love', '40'], ['these were considered to be', 'less'], ['each one has involved the', 'death'], ['symbian phones produced by sony', 'ericsson'], ['any proposals to extend existing', 'arrangements'], ['while not always inventing new', 'technology'], ['although the us leads in', 'this'], ['the band are also working', 'in'], ['they did not tell me', 'which'], ['getting the government even more', 'involved'], ['the canadian music manager who', 'also'], ['as we move from standard', 'definition'], ['after the controversy last year', 'which'], ['the piracy group called itself', 'the'], ['highspeed downlink packet access hsdpa', 'sometimes'], ['earlier in his career melcher', 'had'], ['she called her service provider', 'and'], ['this last category includes the', 'case'], ['among current trends it highlighted', 'the'], ['many other firms have released', 'desktop'], ['the track has become the', 'fastestselling'], ['wendy ide the times film', 'sideways'], ['if the decision on the', 'studio'], ['moreno is also nominated for', 'best'], ['the plates decorating the branches', 'will'], ['the thriller shot straight to', 'the'], ['so what does this mean', 'well'], ['the complaints about the star', 'grabbing'], ['shochiku film studios which was', 'founded'], ['some of the contributing artists', 'are'], ['search engines are often the', 'first'], ['she said she was horrified', 'by'], ['it beats the ipod into', 'second'], ['davis was also a prominent', 'figure'], ['khan was involved in controversy', 'when'], ['more than 120000 trade professionals', 'attended'], ['the next major film award', 'ceremony'], ['the motion picture association of', 'america'], ['mr wong eventually hit upon', 'the'], ['the images have three dimensions', 'and'], ['british tv viewers lead the', 'trend'], ['mainstream radio stations were not', 'going'], ['he wrote ring of fire', 'with'], ['william hill made the move', 'after']], 10: [['highprofile web portals such as google yahoo ebay and amazon', 'are'], ['nbc alleged that boxing regulations had been violated but failed', 'in'], ['equally it may mean hollywood studios ask for more to', 'be'], ['studies have shown that organisations working for record labels and', 'movie'], ['but what has not been explored is how authentication technology', 'may'], ['from 1 january a new law is being introduced to', 'protect'], ['in the second ms love 40 pleaded guilty to possessing', 'a'], ['symbian phones produced by sony ericsson motorola benq arima and', 'fujitsu'], ['any proposals to extend existing arrangements to cover new services', 'would'], ['while not always inventing new technology itself sony was never', 'afraid'], ['although the us leads in this technology at the moment', 'it'], ['the band are also working in the studio on new', 'material'], ['they did not tell me which pictures they had a', 'problem'], ['getting the government even more involved would only waste money', 'that'], ['the canadian music manager who also managed tommy hunter and', 'the'], ['as we move from standard definition video images to highdefinition', 'images'], ['after the controversy last year which saw cbs fined a', 'record'], ['the piracy group called itself the underground network and membership', 'of'], ['highspeed downlink packet access hsdpa sometimes called super 3g will', 'be'], ['earlier in his career melcher had hits as part of', 'duo'], ['she called her service provider and they told her it', 'would'], ['this last category includes the case of a man who', 'became'], ['among current trends it highlighted the final broadcasts of veteran', 'newscasters'], ['the track has become the fastestselling single of the year', 'so'], ['wendy ide the times film sideways director alexander payne actor', 'don'], ['if the decision on the studio side is that they', 'are'], ['the plates decorating the branches will be auctioned off for', 'the'], ['the thriller shot straight to the number one spot after', 'taking'], ['so what does this mean well football manager is the', 'spiritual'], ['the complaints about the star grabbing his crotch were upheld', 'by'], ['shochiku film studios which was founded 110 years ago will', 'become'], ['some of the contributing artists are students or recent graduates', 'of'], ['search engines are often the first port of call for', 'users'], ['she said she was horrified by the deaths of charlene', 'ellis'], ['it beats the ipod into second place in the top', 'ten'], ['davis was also a prominent figure in the civil rights', 'movement'], ['khan was involved in controversy when topless photos alleged to', 'be'], ['more than 120000 trade professionals attended ces in las vegas', 'which'], ['the next major film award ceremony of the season is', 'the'], ['the motion picture association of america wants to stop people', 'using'], ['mr wong eventually hit upon the idea of translating individual', 'colours'], ['the images have three dimensions and i had to find', 'a'], ['british tv viewers lead the trend of illegally downloading us', 'shows'], ['mainstream radio stations were not going to play this kind', 'of'], ['he wrote ring of fire with june carter cash the', 'future'], ['william hill made the move after four people tried to', 'place']], 15: [['highprofile web portals such as google yahoo ebay and amazon are gathering links that lead', 'people'], ['nbc alleged that boxing regulations had been violated but failed in their attempt to have', 'the'], ['equally it may mean hollywood studios ask for more to be put onto the average', 'dvd'], ['studies have shown that organisations working for record labels and movie studios have worked to', 'undermine'], ['but what has not been explored is how authentication technology may potentially play a role', 'in'], ['from 1 january a new law is being introduced to protect computer users from software', 'known'], ['in the second ms love 40 pleaded guilty to possessing a forged prescription and painkiller', 'oxycodone'], ['symbian phones produced by sony ericsson motorola benq arima and fujitsu are unaffected by the', 'skulls'], ['any proposals to extend existing arrangements to cover new services would be for government to', 'consider'], ['while not always inventing new technology itself sony was never afraid to innovate around various', 'formats'], ['although the us leads in this technology at the moment it is also a concern', 'that'], ['the band are also working in the studio on new material their first since the', '1982'], ['getting the government even more involved would only waste money that could be put to', 'better'], ['the canadian music manager who also managed tommy hunter and the statler brothers had been', 'in'], ['as we move from standard definition video images to highdefinition images we have a much', 'greater'], ['after the controversy last year which saw cbs fined a record 550000 £292000 by federal', 'regulators'], ['the piracy group called itself the underground network and membership of it demanded that users', 'share'], ['highspeed downlink packet access hsdpa sometimes called super 3g will be vital for profitable services', 'like'], ['earlier in his career melcher had hits as part of duo called bruce terry with', 'future'], ['she called her service provider and they told her it would be five days before', 'an'], ['this last category includes the case of a man who became so mad with his', 'malfunctioning'], ['among current trends it highlighted the final broadcasts of veteran newscasters tom brokaw barbara walters', 'and'], ['the track has become the fastestselling single of the year so far shifting more copies', 'than'], ['if the decision on the studio side is that they are going to put a', 'lot'], ['the thriller shot straight to the number one spot after taking 22m £117m at the', 'box'], ['so what does this mean well football manager is the spiritual successor to the cm', 'series'], ['shochiku film studios which was founded 110 years ago will become the first cinematic institution', 'to'], ['some of the contributing artists are students or recent graduates of the royal college of', 'art'], ['search engines are often the first port of call for users when they go onto', 'the'], ['she said she was horrified by the deaths of charlene ellis 18 and letisha shakespeare', '17'], ['it beats the ipod into second place in the top ten essentials list which predicts', 'what'], ['davis was also a prominent figure in the civil rights movement and was a voice', 'for'], ['khan was involved in controversy when topless photos alleged to be of her appeared in', 'a'], ['more than 120000 trade professionals attended ces in las vegas which officially ran from 6', 'to'], ['the next major film award ceremony of the season is the bafta awards which take', 'place'], ['the motion picture association of america wants to stop people using the program bittorrent to', 'swap'], ['mr wong eventually hit upon the idea of translating individual colours into music and enlisted', 'the'], ['the images have three dimensions and i had to find a way of reading them', 'myself'], ['british tv viewers lead the trend of illegally downloading us shows from the net according', 'to'], ['mainstream radio stations were not going to play this kind of music and starting their', 'own'], ['he wrote ring of fire with june carter cash the future wife of johnny cash', 'who'], ['william hill made the move after four people tried to place bets on fridays surprise', 'double']], 20: [['highprofile web portals such as google yahoo ebay and amazon are gathering links that lead people to aid and relief', 'organisations'], ['studies have shown that organisations working for record labels and movie studios have worked to undermine kazaa by putting in', 'fakes'], ['but what has not been explored is how authentication technology may potentially play a role in decentralised online communities the', 'key'], ['although the us leads in this technology at the moment it is also a concern that is being raised in', 'europe'], ['the canadian music manager who also managed tommy hunter and the statler brothers had been in declining health according to', 'his'], ['as we move from standard definition video images to highdefinition images we have a much greater need for storage richard', 'doherty'], ['after the controversy last year which saw cbs fined a record 550000 £292000 by federal regulators super bowl organisers had', 'turned'], ['the piracy group called itself the underground network and membership of it demanded that users share between one and 100', 'gigabytes'], ['highspeed downlink packet access hsdpa sometimes called super 3g will be vital for profitable services like mobile internet browsing and', 'mobile'], ['earlier in his career melcher had hits as part of duo called bruce terry with future beach boy bruce johnston', 'which'], ['she called her service provider and they told her it would be five days before an engineer would show up', 'with'], ['this last category includes the case of a man who became so mad with his malfunctioning laptop that he threw', 'it'], ['among current trends it highlighted the final broadcasts of veteran newscasters tom brokaw barbara walters and the impending retirement of', 'cbs'], ['the track has become the fastestselling single of the year so far shifting more copies than the rest of the', 'top'], ['if the decision on the studio side is that they are going to put a lot more on these disks', 'it'], ['so what does this mean well football manager is the spiritual successor to the cm series and it has been', 'released'], ['shochiku film studios which was founded 110 years ago will become the first cinematic institution to receive the berlinale camera', 'award'], ['some of the contributing artists are students or recent graduates of the royal college of art and other leading art', 'colleges'], ['it beats the ipod into second place in the top ten essentials list which predicts what gadgetlovers are likely to', 'covet'], ['the next major film award ceremony of the season is the bafta awards which take place at londons odeon leicester', 'square'], ['mr wong eventually hit upon the idea of translating individual colours into music and enlisted the help of a computer', 'graphics'], ['the images have three dimensions and i had to find a way of reading them myself mr wong told the', 'bbc'], ['mainstream radio stations were not going to play this kind of music and starting their own broadcast station was economically', 'impossible'], ['he wrote ring of fire with june carter cash the future wife of johnny cash who went on to score', 'his']]}


final_acc={}
print(len(dataset[10]))
average_accuracy = 0
num = 0
for line in dataset[10]:
    prefix=line[0]
    expected=line[-1]
    print(prefix)
    result=gpt2.generate(sess,
                length=1,
                temperature=1,
                prefix=prefix,
                return_as_list=True,
                )
    print(result)
    r=str(result[0][len(prefix):]).strip().rstrip("\n")
    #print(r)
    if len(r)!=0:
        x = nlp(r)
        y = nlp(str(expected))
        print(x[0].has_vector)
        #print(y)
        if (x[0].has_vector) and (y[0].has_vector):
            average_accuracy += nlp(r).similarity(nlp(str(expected)))
            num += 1
average_accuracy /= num
print('accuracy：',average_accuracy)
#final_acc[3]=average_accuracy


#{1: [['highprofile', 'web'], ['nbc', 'alleged'], ['equally', 'it'], ['studies', 'have'], ['but', 'what'], ['from', '1'], ['in', 'the'], ['these', 'were'], ['each', 'one'], ['symbian', 'phones'], ['any', 'proposals'], ['while', 'not'], ['although', 'the'], ['the', 'band'], ['they', 'did'], ['getting', 'the'], ['the', 'canadian'], ['as', 'we'], ['after', 'the'], ['the', 'piracy'], ['highspeed', 'downlink'], ['earlier', 'in'], ['she', 'called'], ['this', 'last'], ['among', 'current'], ['many', 'other'], ['the', 'track'], ['wendy', 'ide'], ['if', 'the'], ['moreno', 'is'], ['the', 'plates'], ['the', 'thriller'], ['so', 'what'], ['the', 'complaints'], ['shochiku', 'film'], ['some', 'of'], ['search', 'engines'], ['she', 'said'], ['it', 'beats'], ['davis', 'was'], ['khan', 'was'], ['more', 'than'], ['the', 'next'], ['the', 'motion'], ['mr', 'wong'], ['the', 'images'], ['british', 'tv'], ['mainstream', 'radio'], ['he', 'wrote'], ['william', 'hill']], 2: [['highprofile web', 'portals'], ['nbc alleged', 'that'], ['equally it', 'may'], ['studies have', 'shown'], ['but what', 'has'], ['from 1', 'january'], ['in the', 'second'], ['these were', 'considered'], ['each one', 'has'], ['symbian phones', 'produced'], ['any proposals', 'to'], ['while not', 'always'], ['although the', 'us'], ['the band', 'are'], ['they did', 'not'], ['getting the', 'government'], ['the canadian', 'music'], ['as we', 'move'], ['after the', 'controversy'], ['the piracy', 'group'], ['highspeed downlink', 'packet'], ['earlier in', 'his'], ['she called', 'her'], ['this last', 'category'], ['among current', 'trends'], ['many other', 'firms'], ['the track', 'has'], ['wendy ide', 'the'], ['if the', 'decision'], ['moreno is', 'also'], ['the plates', 'decorating'], ['the thriller', 'shot'], ['so what', 'does'], ['the complaints', 'about'], ['shochiku film', 'studios'], ['some of', 'the'], ['search engines', 'are'], ['she said', 'she'], ['it beats', 'the'], ['davis was', 'also'], ['khan was', 'involved'], ['more than', '120000'], ['the next', 'major'], ['the motion', 'picture'], ['mr wong', 'eventually'], ['the images', 'have'], ['british tv', 'viewers'], ['mainstream radio', 'stations'], ['he wrote', 'ring'], ['william hill', 'made']], 3: [['highprofile web portals', 'such'], ['nbc alleged that', 'boxing'], ['equally it may', 'mean'], ['studies have shown', 'that'], ['but what has', 'not'], ['from 1 january', 'a'], ['in the second', 'ms'], ['these were considered', 'to'], ['each one has', 'involved'], ['symbian phones produced', 'by'], ['any proposals to', 'extend'], ['while not always', 'inventing'], ['although the us', 'leads'], ['the band are', 'also'], ['they did not', 'tell'], ['getting the government', 'even'], ['the canadian music', 'manager'], ['as we move', 'from'], ['after the controversy', 'last'], ['the piracy group', 'called'], ['highspeed downlink packet', 'access'], ['earlier in his', 'career'], ['she called her', 'service'], ['this last category', 'includes'], ['among current trends', 'it'], ['many other firms', 'have'], ['the track has', 'become'], ['wendy ide the', 'times'], ['if the decision', 'on'], ['moreno is also', 'nominated'], ['the plates decorating', 'the'], ['the thriller shot', 'straight'], ['so what does', 'this'], ['the complaints about', 'the'], ['shochiku film studios', 'which'], ['some of the', 'contributing'], ['search engines are', 'often'], ['she said she', 'was'], ['it beats the', 'ipod'], ['davis was also', 'a'], ['khan was involved', 'in'], ['more than 120000', 'trade'], ['the next major', 'film'], ['the motion picture', 'association'], ['mr wong eventually', 'hit'], ['the images have', 'three'], ['british tv viewers', 'lead'], ['mainstream radio stations', 'were'], ['he wrote ring', 'of'], ['william hill made', 'the']], 4: [['highprofile web portals such', 'as'], ['nbc alleged that boxing', 'regulations'], ['equally it may mean', 'hollywood'], ['studies have shown that', 'organisations'], ['but what has not', 'been'], ['from 1 january a', 'new'], ['in the second ms', 'love'], ['these were considered to', 'be'], ['each one has involved', 'the'], ['symbian phones produced by', 'sony'], ['any proposals to extend', 'existing'], ['while not always inventing', 'new'], ['although the us leads', 'in'], ['the band are also', 'working'], ['they did not tell', 'me'], ['getting the government even', 'more'], ['the canadian music manager', 'who'], ['as we move from', 'standard'], ['after the controversy last', 'year'], ['the piracy group called', 'itself'], ['highspeed downlink packet access', 'hsdpa'], ['earlier in his career', 'melcher'], ['she called her service', 'provider'], ['this last category includes', 'the'], ['among current trends it', 'highlighted'], ['many other firms have', 'released'], ['the track has become', 'the'], ['wendy ide the times', 'film'], ['if the decision on', 'the'], ['moreno is also nominated', 'for'], ['the plates decorating the', 'branches'], ['the thriller shot straight', 'to'], ['so what does this', 'mean'], ['the complaints about the', 'star'], ['shochiku film studios which', 'was'], ['some of the contributing', 'artists'], ['search engines are often', 'the'], ['she said she was', 'horrified'], ['it beats the ipod', 'into'], ['davis was also a', 'prominent'], ['khan was involved in', 'controversy'], ['more than 120000 trade', 'professionals'], ['the next major film', 'award'], ['the motion picture association', 'of'], ['mr wong eventually hit', 'upon'], ['the images have three', 'dimensions'], ['british tv viewers lead', 'the'], ['mainstream radio stations were', 'not'], ['he wrote ring of', 'fire'], ['william hill made the', 'move']], 5: [['highprofile web portals such as', 'google'], ['nbc alleged that boxing regulations', 'had'], ['equally it may mean hollywood', 'studios'], ['studies have shown that organisations', 'working'], ['but what has not been', 'explored'], ['from 1 january a new', 'law'], ['in the second ms love', '40'], ['these were considered to be', 'less'], ['each one has involved the', 'death'], ['symbian phones produced by sony', 'ericsson'], ['any proposals to extend existing', 'arrangements'], ['while not always inventing new', 'technology'], ['although the us leads in', 'this'], ['the band are also working', 'in'], ['they did not tell me', 'which'], ['getting the government even more', 'involved'], ['the canadian music manager who', 'also'], ['as we move from standard', 'definition'], ['after the controversy last year', 'which'], ['the piracy group called itself', 'the'], ['highspeed downlink packet access hsdpa', 'sometimes'], ['earlier in his career melcher', 'had'], ['she called her service provider', 'and'], ['this last category includes the', 'case'], ['among current trends it highlighted', 'the'], ['many other firms have released', 'desktop'], ['the track has become the', 'fastestselling'], ['wendy ide the times film', 'sideways'], ['if the decision on the', 'studio'], ['moreno is also nominated for', 'best'], ['the plates decorating the branches', 'will'], ['the thriller shot straight to', 'the'], ['so what does this mean', 'well'], ['the complaints about the star', 'grabbing'], ['shochiku film studios which was', 'founded'], ['some of the contributing artists', 'are'], ['search engines are often the', 'first'], ['she said she was horrified', 'by'], ['it beats the ipod into', 'second'], ['davis was also a prominent', 'figure'], ['khan was involved in controversy', 'when'], ['more than 120000 trade professionals', 'attended'], ['the next major film award', 'ceremony'], ['the motion picture association of', 'america'], ['mr wong eventually hit upon', 'the'], ['the images have three dimensions', 'and'], ['british tv viewers lead the', 'trend'], ['mainstream radio stations were not', 'going'], ['he wrote ring of fire', 'with'], ['william hill made the move', 'after']], 10: [['highprofile web portals such as google yahoo ebay and amazon', 'are'], ['nbc alleged that boxing regulations had been violated but failed', 'in'], ['equally it may mean hollywood studios ask for more to', 'be'], ['studies have shown that organisations working for record labels and', 'movie'], ['but what has not been explored is how authentication technology', 'may'], ['from 1 january a new law is being introduced to', 'protect'], ['in the second ms love 40 pleaded guilty to possessing', 'a'], ['symbian phones produced by sony ericsson motorola benq arima and', 'fujitsu'], ['any proposals to extend existing arrangements to cover new services', 'would'], ['while not always inventing new technology itself sony was never', 'afraid'], ['although the us leads in this technology at the moment', 'it'], ['the band are also working in the studio on new', 'material'], ['they did not tell me which pictures they had a', 'problem'], ['getting the government even more involved would only waste money', 'that'], ['the canadian music manager who also managed tommy hunter and', 'the'], ['as we move from standard definition video images to highdefinition', 'images'], ['after the controversy last year which saw cbs fined a', 'record'], ['the piracy group called itself the underground network and membership', 'of'], ['highspeed downlink packet access hsdpa sometimes called super 3g will', 'be'], ['earlier in his career melcher had hits as part of', 'duo'], ['she called her service provider and they told her it', 'would'], ['this last category includes the case of a man who', 'became'], ['among current trends it highlighted the final broadcasts of veteran', 'newscasters'], ['the track has become the fastestselling single of the year', 'so'], ['wendy ide the times film sideways director alexander payne actor', 'don'], ['if the decision on the studio side is that they', 'are'], ['the plates decorating the branches will be auctioned off for', 'the'], ['the thriller shot straight to the number one spot after', 'taking'], ['so what does this mean well football manager is the', 'spiritual'], ['the complaints about the star grabbing his crotch were upheld', 'by'], ['shochiku film studios which was founded 110 years ago will', 'become'], ['some of the contributing artists are students or recent graduates', 'of'], ['search engines are often the first port of call for', 'users'], ['she said she was horrified by the deaths of charlene', 'ellis'], ['it beats the ipod into second place in the top', 'ten'], ['davis was also a prominent figure in the civil rights', 'movement'], ['khan was involved in controversy when topless photos alleged to', 'be'], ['more than 120000 trade professionals attended ces in las vegas', 'which'], ['the next major film award ceremony of the season is', 'the'], ['the motion picture association of america wants to stop people', 'using'], ['mr wong eventually hit upon the idea of translating individual', 'colours'], ['the images have three dimensions and i had to find', 'a'], ['british tv viewers lead the trend of illegally downloading us', 'shows'], ['mainstream radio stations were not going to play this kind', 'of'], ['he wrote ring of fire with june carter cash the', 'future'], ['william hill made the move after four people tried to', 'place']], 15: [['highprofile web portals such as google yahoo ebay and amazon are gathering links that lead', 'people'], ['nbc alleged that boxing regulations had been violated but failed in their attempt to have', 'the'], ['equally it may mean hollywood studios ask for more to be put onto the average', 'dvd'], ['studies have shown that organisations working for record labels and movie studios have worked to', 'undermine'], ['but what has not been explored is how authentication technology may potentially play a role', 'in'], ['from 1 january a new law is being introduced to protect computer users from software', 'known'], ['in the second ms love 40 pleaded guilty to possessing a forged prescription and painkiller', 'oxycodone'], ['symbian phones produced by sony ericsson motorola benq arima and fujitsu are unaffected by the', 'skulls'], ['any proposals to extend existing arrangements to cover new services would be for government to', 'consider'], ['while not always inventing new technology itself sony was never afraid to innovate around various', 'formats'], ['although the us leads in this technology at the moment it is also a concern', 'that'], ['the band are also working in the studio on new material their first since the', '1982'], ['getting the government even more involved would only waste money that could be put to', 'better'], ['the canadian music manager who also managed tommy hunter and the statler brothers had been', 'in'], ['as we move from standard definition video images to highdefinition images we have a much', 'greater'], ['after the controversy last year which saw cbs fined a record 550000 £292000 by federal', 'regulators'], ['the piracy group called itself the underground network and membership of it demanded that users', 'share'], ['highspeed downlink packet access hsdpa sometimes called super 3g will be vital for profitable services', 'like'], ['earlier in his career melcher had hits as part of duo called bruce terry with', 'future'], ['she called her service provider and they told her it would be five days before', 'an'], ['this last category includes the case of a man who became so mad with his', 'malfunctioning'], ['among current trends it highlighted the final broadcasts of veteran newscasters tom brokaw barbara walters', 'and'], ['the track has become the fastestselling single of the year so far shifting more copies', 'than'], ['if the decision on the studio side is that they are going to put a', 'lot'], ['the thriller shot straight to the number one spot after taking 22m £117m at the', 'box'], ['so what does this mean well football manager is the spiritual successor to the cm', 'series'], ['shochiku film studios which was founded 110 years ago will become the first cinematic institution', 'to'], ['some of the contributing artists are students or recent graduates of the royal college of', 'art'], ['search engines are often the first port of call for users when they go onto', 'the'], ['she said she was horrified by the deaths of charlene ellis 18 and letisha shakespeare', '17'], ['it beats the ipod into second place in the top ten essentials list which predicts', 'what'], ['davis was also a prominent figure in the civil rights movement and was a voice', 'for'], ['khan was involved in controversy when topless photos alleged to be of her appeared in', 'a'], ['more than 120000 trade professionals attended ces in las vegas which officially ran from 6', 'to'], ['the next major film award ceremony of the season is the bafta awards which take', 'place'], ['the motion picture association of america wants to stop people using the program bittorrent to', 'swap'], ['mr wong eventually hit upon the idea of translating individual colours into music and enlisted', 'the'], ['the images have three dimensions and i had to find a way of reading them', 'myself'], ['british tv viewers lead the trend of illegally downloading us shows from the net according', 'to'], ['mainstream radio stations were not going to play this kind of music and starting their', 'own'], ['he wrote ring of fire with june carter cash the future wife of johnny cash', 'who'], ['william hill made the move after four people tried to place bets on fridays surprise', 'double']], 20: [['highprofile web portals such as google yahoo ebay and amazon are gathering links that lead people to aid and relief', 'organisations'], ['studies have shown that organisations working for record labels and movie studios have worked to undermine kazaa by putting in', 'fakes'], ['but what has not been explored is how authentication technology may potentially play a role in decentralised online communities the', 'key'], ['although the us leads in this technology at the moment it is also a concern that is being raised in', 'europe'], ['the canadian music manager who also managed tommy hunter and the statler brothers had been in declining health according to', 'his'], ['as we move from standard definition video images to highdefinition images we have a much greater need for storage richard', 'doherty'], ['after the controversy last year which saw cbs fined a record 550000 £292000 by federal regulators super bowl organisers had', 'turned'], ['the piracy group called itself the underground network and membership of it demanded that users share between one and 100', 'gigabytes'], ['highspeed downlink packet access hsdpa sometimes called super 3g will be vital for profitable services like mobile internet browsing and', 'mobile'], ['earlier in his career melcher had hits as part of duo called bruce terry with future beach boy bruce johnston', 'which'], ['she called her service provider and they told her it would be five days before an engineer would show up', 'with'], ['this last category includes the case of a man who became so mad with his malfunctioning laptop that he threw', 'it'], ['among current trends it highlighted the final broadcasts of veteran newscasters tom brokaw barbara walters and the impending retirement of', 'cbs'], ['the track has become the fastestselling single of the year so far shifting more copies than the rest of the', 'top'], ['if the decision on the studio side is that they are going to put a lot more on these disks', 'it'], ['so what does this mean well football manager is the spiritual successor to the cm series and it has been', 'released'], ['shochiku film studios which was founded 110 years ago will become the first cinematic institution to receive the berlinale camera', 'award'], ['some of the contributing artists are students or recent graduates of the royal college of art and other leading art', 'colleges'], ['it beats the ipod into second place in the top ten essentials list which predicts what gadgetlovers are likely to', 'covet'], ['the next major film award ceremony of the season is the bafta awards which take place at londons odeon leicester', 'square'], ['mr wong eventually hit upon the idea of translating individual colours into music and enlisted the help of a computer', 'graphics'], ['the images have three dimensions and i had to find a way of reading them myself mr wong told the', 'bbc'], ['mainstream radio stations were not going to play this kind of music and starting their own broadcast station was economically', 'impossible'], ['he wrote ring of fire with june carter cash the future wife of johnny cash who went on to score', 'his']]}

        
    
    


46
highprofile web portals such as google yahoo ebay and amazon
['highprofile web portals such as google yahoo ebay and amazon a']
True
nbc alleged that boxing regulations had been violated but failed
['nbc alleged that boxing regulations had been violated but failed to']
True
equally it may mean hollywood studios ask for more to
['equally it may mean hollywood studios ask for more to be']
True
studies have shown that organisations working for record labels and
['studies have shown that organisations working for record labels and gun']
True
but what has not been explored is how authentication technology
['but what has not been explored is how authentication technology can']
True
from 1 january a new law is being introduced to
['from 1 january a new law is being introduced to help']
True
in the second ms love 40 pleaded guilty to possessing
['in the second ms love 40 pleaded guilty to possessing an']
True
symbian phones produced by sony ericsson motorola benq arima and
['symbian phones 

In [15]:
print(final_acc)
#length1: 0.38263516892041544
#length2: 0.47359868940601696
#length3: 0.5158241760324638
#length4: 0.4772309125322724
#length5: 0.5075405871112838
#10:

#20:


{1: 0.38263516892041544}


For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

In [ ]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="The secret of life is",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.